In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pyplot
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import accuracy_score 
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
import pickle
from imblearn.over_sampling import SMOTE

In [2]:
model_data = pd.read_csv("top10_data.csv", delimiter = ",")
model_data.head(3)

,Rank,Title,Artists,Date,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,...,Points (Total),Points (Ind for each Artist/Nat),id,Song URL,Highest Rank,Previous Popularity,Number Songs Previous,Number of Artists,Top 10,Previous Top Rank Avg
0,197,I Miss You,"Grey, Bahari",2017-02-01,-0.166703,0.745431,0.196288,-0.310595,0.528030,-0.167878,...,4,2.0,1sl5tSzfK1JrcQIcA8rYDy,https://open.spotify.com/track/1sl5tSzfK1JrcQI...,175,0.0,0.0,2.0,False,201.0
1,188,Location,Khalid,2017-02-01,0.411958,-1.085786,-1.772413,2.446186,0.375981,-0.167878,...,13,13.0,152lZdxL1OR0ZMW6KquMif,https://open.spotify.com/track/152lZdxL1OR0ZMW...,33,0.0,0.0,1.0,False,201.0
2,199,Magic,"Thomas Gold, Jillian Edwards",2017-02-01,-0.308005,1.431411,0.192504,-0.718699,-0.744384,-0.155927,...,2,1.0,5abwHwUhywLGNli5NDcEN7,https://open.spotify.com/track/5abwHwUhywLGNli...,168,0.0,0.0,2.0,False,201.0


In [3]:
model_data["Top 10"].value_counts()

Top 10
False    6808
True      693
Name: count, dtype: int64

In [4]:
#https://open.spotify.com/track/?si=c5ba6db6d5584b46

In [5]:
ids = ["20uknEOwx3PpiGL2Es2vCg","5eL45g9qvFvydEYcmv4zic","6mxsd0LQt2qk2rpLEqlCJw","5bD6mFl116xCxVMLy7IK9F",
       "5hT3U17QZiP74YN9xF32JG","6afdNrotJ1PCt9DoFiHpLj","01QPvRIYC5QG9E3khE6P9M","3xY5aYJcdvZX9xQxNd02ku",
       "1ytEV08vvKh6MtUaVP9siI", "4OMJGnvZfDvsePyCwRGO7X", "1v03tMJZRX2eN4dM89pCfS"]

In [6]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json

In [7]:
#credentials = json.load(open('credentials.json'))
client_id = "7d89129b25794e769bca663ca259d520"
client_secret = "165d7ab63dcf4752b71d2b741ebb8935"

In [8]:
client_credentials_manager = SpotifyClientCredentials(
    client_id=client_id,
    client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [9]:
track_id = '4NUmZ5svM1Bbmaz5udqUeJ'

# Get audio features for the track
audio_features = sp.audio_features([track_id])

# Print the audio features
print(audio_features)

[{'danceability': 0.697, 'energy': 0.732, 'key': 11, 'loudness': -6.534, 'mode': 0, 'speechiness': 0.0646, 'acousticness': 0.0139, 'instrumentalness': 0.000631, 'liveness': 0.0807, 'valence': 0.651, 'tempo': 124.932, 'type': 'audio_features', 'id': '4NUmZ5svM1Bbmaz5udqUeJ', 'uri': 'spotify:track:4NUmZ5svM1Bbmaz5udqUeJ', 'track_href': 'https://api.spotify.com/v1/tracks/4NUmZ5svM1Bbmaz5udqUeJ', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4NUmZ5svM1Bbmaz5udqUeJ', 'duration_ms': 170253, 'time_signature': 4}]


In [10]:
columns = ['song_name', 'artist', 'Danceability', 'Energy', 'Loudness', 'Speechiness', 'Acousticness', 'Instrumentalness', 
           'Valence', 'id', 'analysis_url']
newSongs = pd.DataFrame(columns=columns)


for track_id in ids:
    # Retrieve track details and audio features
    track_info = sp.track(track_id)
    audio_features = sp.audio_features([track_id])[0]
    
    #Used chatgpt to quickly generate code to get the relevant audio features from the dictionary
    track_info = {
        'song_name': track_info['name'],
        'artist': ', '.join([artist['name'] for artist in track_info['artists']]),
        'Danceability': audio_features['danceability'],
        'Energy': audio_features['energy'],
        'Loudness': audio_features['loudness'],
        'Speechiness': audio_features['speechiness'],
        'Acousticness': audio_features['acousticness'],
        'Instrumentalness': audio_features['instrumentalness'],
        'Valence': audio_features['valence'],
        'id': audio_features['id'],
        'analysis_url': audio_features['analysis_url']
    }
    
    # Append the track_info to the DataFrame using concat
    newSongs = pd.concat([newSongs, pd.DataFrame([track_info])], ignore_index=True)

    

In [11]:
newSongs.head(3)

,song_name,artist,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Valence,id,analysis_url
0,Magical,Ed Sheeran,0.658,0.632,-8.492,0.0298,0.748,0.000073,0.464,20uknEOwx3PpiGL2Es2vCg,https://api.spotify.com/v1/audio-analysis/20uk...
1,England,Ed Sheeran,0.622,0.666,-10.786,0.0423,0.641,0.000086,0.520,5eL45g9qvFvydEYcmv4zic,https://api.spotify.com/v1/audio-analysis/5eL4...
2,Different Kind Of Beautiful,Alec Benjamin,0.522,0.427,-7.419,0.0602,0.709,0.000004,0.644,6mxsd0LQt2qk2rpLEqlCJw,https://api.spotify.com/v1/audio-analysis/6mxs...


In [12]:
results = []

# Iterate over each row in 'newSongs'
for index, row in newSongs.iterrows():
    target_artist = row['artist']

    # Check if there are any rows that match the condition
    matching_rows = model_data[model_data['Artists'].str.contains(target_artist)]

    if not matching_rows.empty:
        # Find the last occurrence of the target artist
        last_occurrence = matching_rows.index[-1]

        # Get relevant information from model_data
        previous_popularity = model_data.at[last_occurrence, 'Previous Popularity']
        #add number songs previous because last song in Model_csv would not have been considered and now should be
        number_songs_previous = model_data.at[last_occurrence, 'Number Songs Previous'] + 1
        #add previous top rank average
        previous_top_rank_avg = model_data.at[last_occurrence, "Previous Top Rank Avg"]

        #account for edge case where artist had only one song appear in top charts and it reached top 10
        results.append((previous_popularity, number_songs_previous, previous_top_rank_avg))
    else:
        #append 0 as placeholder for previous pop and number songs previous. append 201 for previous top rank avg
        previous_popularity = 0.0
        number_songs_previous = 0.0
        previous_top_rank_avg = 201.0
        results.append((previous_popularity, number_songs_previous, previous_top_rank_avg))

#add a new column to 'newSongs' with the results
newSongs[['Previous Popularity', 'Number Songs Previous', "Previous Top Rank Avg"]] = pd.DataFrame(results)


In [13]:
#find number of artists in the new songs separating by ','
newSongs['Number of Artists'] = newSongs['artist'].str.count(',') + 1

#get numerical features and apply standard scaler to them
numerical_features = newSongs.columns[2:9]
with open('scaler_numerical.pkl', 'rb') as f:
    scaler = pickle.load(f)
    newSongs_scaled_numerical = scaler.fit_transform(newSongs[numerical_features])

#add in the scaled data to df and reset index
newSongs.loc[:, numerical_features] = newSongs_scaled_numerical
newSongs = newSongs.reset_index(drop = True)
newSongs

,song_name,artist,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Valence,id,analysis_url,Previous Popularity,Number Songs Previous,Previous Top Rank Avg,Number of Artists
0,Magical,Ed Sheeran,-0.216151,-0.369640,-1.207869,-1.072420,1.729470,-0.354672,-0.970927,20uknEOwx3PpiGL2Es2vCg,https://api.spotify.com/v1/audio-analysis/20uk...,1.0,68.0,54.865672,1
1,England,Ed Sheeran,-0.555816,-0.068452,-2.469014,-0.673778,1.359451,-0.351102,-0.574315,5eL45g9qvFvydEYcmv4zic,https://api.spotify.com/v1/audio-analysis/5eL4...,1.0,68.0,54.865672,1
2,Different Kind Of Beautiful,Alec Benjamin,-1.499331,-2.185629,-0.617978,-0.102922,1.594603,-0.373748,0.303898,6mxsd0LQt2qk2rpLEqlCJw,https://api.spotify.com/v1/audio-analysis/6mxs...,0.0,1.0,201.000000,1
3,GATITA GANGSTER,"KAROL G, Dei V",1.368954,-1.078319,0.620077,2.569576,-0.279699,-0.373095,-0.489327,5bD6mFl116xCxVMLy7IK9F,https://api.spotify.com/v1/audio-analysis/5bD6...,0.0,0.0,201.000000,2
4,"Better Off (Alone, Pt. III)","Alan Walker, Dash Berlin, Vikkstar",-1.612553,0.153010,0.602485,-0.590860,-0.752079,-0.374806,-1.594175,5hT3U17QZiP74YN9xF32JG,https://api.spotify.com/v1/audio-analysis/5hT3...,0.0,0.0,201.000000,3
5,Real Love,"Martin Garrix, Lloyiso",-0.310502,0.702236,0.415567,-0.342108,-0.757958,-0.374806,-1.041751,6afdNrotJ1PCt9DoFiHpLj,https://api.spotify.com/v1/audio-analysis/6afd...,0.0,0.0,201.000000,2
6,Como Yo :(,"Marshmello, Tiago PZK",0.661318,-0.847998,-0.185868,0.043778,0.176773,-0.374394,0.488039,01QPvRIYC5QG9E3khE6P9M,https://api.spotify.com/v1/audio-analysis/01QP...,0.0,0.0,201.000000,2
7,In The City,"Charli XCX, Sam Smith",0.519791,0.711094,0.460647,-0.609995,-0.743088,-0.210278,0.332227,3xY5aYJcdvZX9xQxNd02ku,https://api.spotify.com/v1/audio-analysis/3xY5...,0.0,0.0,201.000000,2
8,Kalash (feat. BEAM),"Delara, BEAM",-0.527511,0.657944,0.849326,1.325812,-0.782856,-0.374806,0.332227,1ytEV08vvKh6MtUaVP9siI,https://api.spotify.com/v1/audio-analysis/1ytE...,0.0,0.0,201.000000,2
9,Houdini,Dua Lipa,0.595272,1.021141,0.780057,-0.141192,-0.844757,0.020721,1.876182,4OMJGnvZfDvsePyCwRGO7X,https://api.spotify.com/v1/audio-analysis/4OMJ...,1.0,21.0,41.400000,1


In [14]:
#define features and label. Song characteristics is the numerical features
features = ['Danceability', 'Energy', 'Loudness', 'Speechiness', 'Acousticness', 'Instrumentalness',
       'Valence', 'Previous Popularity', "Number of Artists", 'Number Songs Previous', 'Previous Top Rank Avg']
label = ['Top 10']

X = model_data[features]
y = model_data[label]

#use SMOTE to oversample the minority class in the training set

X = X.values
y = y.values.reshape(-1)

#split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


#use SMOTE to oversample the minority class in the training set
smote = SMOTE(sampling_strategy=1, random_state=42)
X_train,y_train = smote.fit_resample(X_train,y_train)


#use SMOTE to oversample the minority class in the test set
smote = SMOTE(sampling_strategy=1, random_state=42)
X_test,y_test = smote.fit_resample(X_test,y_test)



In [15]:
#run on test for best parameters, this model follows from modelCreation
xgbClass_tuned = XGBClassifier( 
                        subsample=1,
                        colsample_bytree=0.9,
                        n_estimators = 200,
                        learning_rate = 0.1,
                        max_depth = 3,
                        objective='binary:logistic',  
                        eval_metric='logloss',
                        random_state = 42
                    )

                    #fit the model to your training data
xgbClass_tuned.fit(X_train, y_train)

#make predictions on the test data
y_pred = xgbClass_tuned.predict(X_test)
f1 = f1_score(y_test, y_pred)
y_true = y_test

#calculate test statistics
y_probas = xgbClass_tuned.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_true, y_probas)
accuracy = accuracy_score(y_test, y_pred)
feature_importances = xgbClass_tuned.feature_importances_
feature_importances

#print out relevant info on model performance
print("Accuracy: " + str(accuracy))
print("ROC_AUC: " + str(roc_auc))
print("F1: " + str(f1))
print("-------------")
print("Feature Importance")
for i, feat_imp in enumerate(feature_importances):
    print(features[i] + ": " + str(feat_imp))

print("-------------")

Accuracy: 0.7863436123348018
ROC_AUC: 0.8759590069022621
F1: 0.7776928953399542
-------------
Feature Importance
Danceability: 0.032140166
Energy: 0.03173845
Loudness: 0.046205472
Speechiness: 0.042656604
Acousticness: 0.03271086
Instrumentalness: 0.06670809
Valence: 0.021607418
Previous Popularity: 0.4473065
Number of Artists: 0.12503499
Number Songs Previous: 0.056239285
Previous Top Rank Avg: 0.09765223
-------------


In [16]:
#print out predictions for if a new song will be in the top 10 or not be in the top 10
newSongs_X = newSongs[features]
predictions = []
for i, row in newSongs_X.iterrows():
    future_pred = xgbClass_tuned.predict(np.array(row).reshape(1,-1))
    future_pred_boolean = bool(future_pred)
    predictions.append(future_pred_boolean)
    
predictions

[False, False, False, False, False, True, True, True, False, True, False]

In [17]:
#determine the probability liklihood for each song being either top 10 or not top 10
probs = xgbClass_tuned.predict_proba(newSongs_X)
probs

array([[0.8084658 , 0.19153424],
       [0.79352826, 0.20647174],
       [0.9432557 , 0.05674426],
       [0.5745438 , 0.42545623],
       [0.73553205, 0.26446792],
       [0.45842332, 0.5415767 ],
       [0.410886  , 0.589114  ],
       [0.3897164 , 0.6102836 ],
       [0.5309108 , 0.4690892 ],
       [0.16253996, 0.83746004],
       [0.81498206, 0.18501793]], dtype=float32)

In [18]:
#segment out the two songs of interest for the report
newSongs.iloc[[5, 9]]

,song_name,artist,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Valence,id,analysis_url,Previous Popularity,Number Songs Previous,Previous Top Rank Avg,Number of Artists
5,Real Love,"Martin Garrix, Lloyiso",-0.310502,0.702236,0.415567,-0.342108,-0.757958,-0.374806,-1.041751,6afdNrotJ1PCt9DoFiHpLj,https://api.spotify.com/v1/audio-analysis/6afd...,0.0,0.0,201.0,2
9,Houdini,Dua Lipa,0.595272,1.021141,0.780057,-0.141192,-0.844757,0.020721,1.876182,4OMJGnvZfDvsePyCwRGO7X,https://api.spotify.com/v1/audio-analysis/4OMJ...,1.0,21.0,41.4,1
